In [242]:
from datetime import datetime
starting = datetime.now()

In [243]:
!pip install -q optbinning

In [244]:
import os
try:
    from sklearn.feature_selection import chi2,mutual_info_classif
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import f1_score, classification_report,accuracy_score
    from sklearn.naive_bayes import CategoricalNB,GaussianNB,BernoulliNB,ComplementNB,MultinomialNB
    from sklearn.preprocessing import LabelEncoder,QuantileTransformer,StandardScaler,OneHotEncoder,OrdinalEncoder
    import itertools
    import matplotlib.pyplot as plt
    import numpy as np # linear algebra
    import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
    import seaborn as sns
except:
    os.system("pip install -q scikit-learn")
    os.system("pip install -q numpy")
    os.system("pip install -q matplotlib")
    os.system("pip install -q seaborn")
    os.system("pip install -q pandas")
    from sklearn.feature_selection import chi2
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import f1_score, classification_report,accuracy_score
    from sklearn.naive_bayes import CategoricalNB,GaussianNB,BernoulliNB,ComplementNB,MultinomialNB
    from sklearn.preprocessing import LabelEncoder,QuantileTransformer,StandardScaler,OneHotEncoder,OrdinalEncoder
    import itertools
    import matplotlib.pyplot as plt
    import numpy as np # linear algebra
    import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
    import seaborn as sns


In [245]:
def kom_kol(a_list):
    all_combinations = []
    for r in range(len(a_list) + 1):
        combinations_object = itertools.combinations(a_list, r)
        combinations_list = list(combinations_object)
        all_combinations += combinations_list
    fitur_list = []
    for i in all_combinations:
        cek = list(i)
        if(cek != []):
            fitur_list.append(cek)
    return(fitur_list)

In [246]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# Dasarnya

dengan melakukan feature engineering dengan bantuan link ini https://machinelearningmastery.com/better-naive-bayes/

In [247]:
df = pd.read_csv("../input/covid19-symptoms-dataset/covid-19 symptoms dataset.csv")
df = df.drop_duplicates()
print(df.shape)

n4 = 212
df = df.sample(n=n4,random_state=42)

df_all = pd.read_csv("../input/covid19-symptoms-dataset/covid-19 symptoms dataset.csv")
df_all = df_all.drop_duplicates()

print(df.shape)
df.head()

# EDA

In [248]:
sns.catplot(x="bodyPain",hue="infectionProb",kind="count",data=df_all,ci=None) #["fever","bodyPain","age","runnyNose","diffBreath",'infectionProb]

In [249]:
sns.catplot(x="runnyNose",hue="infectionProb",kind="count",data=df_all,ci=None)

In [250]:
sns.catplot(x="diffBreath",hue="infectionProb",kind="count",data=df_all,ci=None)

In [251]:
sns.catplot(x="fever",hue="infectionProb",kind="count",data=df_all,ci=None)

In [252]:
cek_no_covid = df_all.groupby(['age','infectionProb'])['fever'].count().loc[:,0]
cek_yes_covid = df_all.groupby(['age','infectionProb'])['fever'].count().loc[:,1]
plt.figure(figsize=(20,5))
plt.bar(cek_no_covid.index,cek_no_covid,label='tidak covid')
plt.bar(cek_yes_covid.index,cek_yes_covid,label='covid')
plt.legend()

In [253]:
df_all.head()

In [254]:
df_all

In [255]:
#Tanpa Sampling
import statsmodels.api as sm

X_stat_reg = df_all[df_all.drop(['infectionProb'],axis=1).columns]
y_stat_reg = df_all[['infectionProb']]

log_reg = sm.Logit(y_stat_reg, X_stat_reg).fit()
log_reg.summary()

In [256]:
# dengan sampling
import statsmodels.api as sm

X_stat_reg = df[df.drop(['infectionProb'],axis=1).columns]
y_stat_reg = df[['infectionProb']]

log_reg = sm.Logit(y_stat_reg, X_stat_reg).fit()
log_reg.summary()

# check Independency 

In [257]:
#bodyPain	age	runnyNose	diffBreath
selection = ['bodyPain',"runnyNose","diffBreath"]
for i in range(len(selection)):
    for j in range((i+1),len(selection)):
        if(i!=j):
            has = chi2(df_all[[selection[i]]],df_all[[selection[j]]])
            print("nilai p_value ",selection[i],"dan",selection[j],"adalah",has[1][0])

Dengan sampling , naive bayes seperlunya tuh tidak perlu data yang terlalu banyak

# preprocessing

In [258]:
from optbinning import OptimalBinning
optb = OptimalBinning(name='age', dtype="numerical", solver="mip")

x = df.age.values
y = df.infectionProb
optb.fit(x, y)
optb.splits

Binning dapat meningkatkan akurasi

In [259]:
batasan_umur = optb.splits.tolist()
binning = [0]
for i in batasan_umur:
    if(isinstance(x, str)):
        cek = int(i[4:6])
    else:
        cek = i
    binning.append(cek)
binning.append(100)
binning = sorted(binning)
labelku = [i for i in range(len(batasan_umur)+1)]

df['aging'] = pd.cut(df['age'],bins=binning,labels=labelku)
df.drop('age',axis=1,inplace=True)
df = pd.get_dummies(df,columns=['aging','diffBreath'])

df_all['aging'] = pd.cut(df_all['age'],bins=binning,labels=labelku)
df_all.drop('age',axis=1,inplace=True)
df_all = pd.get_dummies(df_all,columns=['aging','diffBreath'])

df.head()

## interval age 
1. aging_0 = 0  -  24 tahun
2. aging_1 = 25 -  37 tahun
3. aging_2 = 38 -  44 tahun
4. aging_3 = 45 -  93 tahun
5. aging_4 = 94 - 100 tahun

In [260]:
df = df.drop_duplicates()
X = df.drop("infectionProb",axis=1)
y = df["infectionProb"]

print(df.shape)

df_all = df_all.drop_duplicates()
X_allin = df_all.drop("infectionProb",axis=1)
y_allin = df_all["infectionProb"]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

model = GaussianNB()
model.fit(X_train,y_train)
y_predict = model.predict(X_test)
f_score = accuracy_score(y_test, y_predict)
print("fiturnya",X.columns)
f_score

dengan seluruh feature, nilanya 0.72222222

In [261]:
agingku = [('aging_'+str(i)) for i in range(5) ]
listku = [df[i].sum() for i in agingku]
print(listku)

agingku_0 = df.loc[df['aging_0']==1,'infectionProb']
agingku_1 = df.loc[df['aging_1']==1,'infectionProb']
agingku_2 = df.loc[df['aging_2']==1,'infectionProb']
agingku_3 = df.loc[df['aging_3']==1,'infectionProb']
agingku_4 = df.loc[df['aging_4']==1,'infectionProb']


plt.bar(agingku,listku)

terinfeksi = [agingku_0[agingku_0==0].count(),agingku_1[agingku_1==0].count(),agingku_2[agingku_2==0].count(),
       agingku_3[agingku_3==0].count(),agingku_4[agingku_4==0].count()]
tidak_terinfeksi = (np.array(listku) - np.array(terinfeksi)).tolist()
print(terinfeksi)
print(tidak_terinfeksi)


width = 0.35


x = np.arange(len(listku))

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, terinfeksi, width, label='terinfeksi')
rects2 = ax.bar(x + width/2, tidak_terinfeksi, width, label='tidak_terinfeksi')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('banyak orang')
ax.set_ylabel('kategori umur')
ax.set_title('banyak orang terinfeksi berdasar umur')
ax.legend()
ax.set_xticks(x)
ax.set_xticklabels(agingku)


ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()

In [262]:
l = []
for i in X_train.columns:
    feat_chos = i
    model.fit(X_train[[feat_chos]],y_train)
    y_predict = model.predict(X_test[[feat_chos]])
    f_score = accuracy_score(y_test, y_predict)
    if(f_score>=0.5):
        print("myscore only", i , f_score)
        l.append(i)
print(l)

In [263]:
X = df[l]
y = df["infectionProb"]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
model = GaussianNB()
model.fit(X_train,y_train)
y_predict = model.predict(X_test)
f_score = accuracy_score(y_test, y_predict)

print("dengan samplingnya yakni n = 212 maka akurasinya",f_score)
print("jumlah barisnya sebanyak",X_test.shape[0],"baris")
print('fiturnya',l)

# Tanpa Sampling

In [264]:
df2 = pd.read_csv("../input/covid19-symptoms-dataset/covid-19 symptoms dataset.csv")
df2 = df2.drop_duplicates()
df2

In [265]:
sns.histplot(x='age',data=df2)

In [266]:
sns.catplot(x="bodyPain",hue="infectionProb",kind="count",data=df2,ci=None)

In [267]:
sns.catplot(x="runnyNose",hue="infectionProb",kind="count",data=df2,ci=None)

In [268]:
sns.catplot(x="diffBreath",hue="infectionProb",kind="count",data=df2,ci=None)

In [269]:
sns.catplot(x="fever",hue="infectionProb",kind="count",data=df_all,ci=None)

In [270]:
x = df2.age.values
y = df2.infectionProb
optb.fit(x, y)

batasan_umur = optb.splits.tolist()
binning = [0]
for i in batasan_umur:
    if(isinstance(x, str)):
        cek = int(i[4:6])
    else:
        cek = i
    binning.append(cek)
binning.append(100)
binning = sorted(binning)
labelku = [i for i in range(len(batasan_umur)+1)]

df2['aging'] = pd.cut(df2['age'],bins=binning,labels=labelku)
df2.drop('age',axis=1,inplace=True)
df2 = pd.get_dummies(df2,columns=['aging','diffBreath','fever'])

df2.head()

In [271]:
# X_ku = df2.drop("infectionProb",axis=1)
# y_ku = df2["infectionProb"]

# starting = datetime.now()

# com_col = kom_kol(X_ku.columns.values)
# score = []
# c = 0
# for i in com_col:
#     X_train,X_test,y_train,y_test = train_test_split(X_ku[i],y_ku,test_size=0.2,random_state=42)
#     model = BernoulliNB()
#     model.fit(X_train,y_train)
#     y_predict = model.predict(X_test)
#     f_score = accuracy_score(y_test, y_predict)
#     score.append(f_score)
#     c+=1
#     if(c % 100000 == 0 ):
#         ending = datetime.now()
#         interval = ending - starting
#         interval = interval.total_seconds()
#         jam = interval // 3600
#         interval %= 3600
#         menit = interval // 60
#         interval %= 60
#         detik = interval
#         print("{} durasinya {} jam {} menit {} detik".format(c,jam,menit,detik))

# print(score.index(max(score)))
# print(com_col[score.index(max(score))])

['bodyPain', 'aging_3', 'aging_4', 'diffBreath_1', 'fever_99', 'fever_102']


binning<br>
aging 3 = 38 - 51<br>
aging 4 = 52 - 67<br>

In [272]:
binning

In [273]:
df2 = pd.read_csv("../input/covid19-symptoms-dataset/covid-19 symptoms dataset.csv")
df2 = df2.drop_duplicates()
df2['diffBreath'] += 1
df2.head()

In [274]:
x = df2.age.values
y = df2.infectionProb
optb.fit(x, y)

batasan_umur = optb.splits.tolist()

print("batasan umurnya adalah",batasan_umur)
binning = [0]
for i in batasan_umur:
    if(isinstance(x, str)):
        cek = int(i[4:6])
    else:
        cek = i
    binning.append(cek)
binning.append(100)
binning = sorted(binning)
labelku = [i for i in range(len(batasan_umur)+1)]

df2['aging'] = pd.cut(df2['age'],bins=binning,labels=labelku)
df2.drop('age',axis=1,inplace=True)
# df2 = pd.get_dummies(df2,columns=['aging'])
df2.head()

In [275]:
X_ku = df2.drop("infectionProb",axis=1)
y_ku = df2["infectionProb"]

starting = datetime.now()

com_col = kom_kol(X_ku.columns.values)

score_bernoulli = []
score_multi     = []
score_gaussian  = []
c = 0
for i in com_col:
    X_train,X_test,y_train,y_test = train_test_split(X_ku[i],y_ku,test_size=0.2,random_state=42)
    
    model = MultinomialNB()
    model.fit(X_train,y_train)
    y_predict = model.predict(X_test)
    f_score = accuracy_score(y_test, y_predict)
    score_multi.append(f_score)
    
    model = BernoulliNB()
    model.fit(X_train,y_train)
    y_predict = model.predict(X_test)
    f_score = accuracy_score(y_test, y_predict)
    score_bernoulli.append(f_score)
    
    model = GaussianNB()
    model.fit(X_train,y_train)
    y_predict = model.predict(X_test)
    f_score = accuracy_score(y_test, y_predict)
    score_gaussian.append(f_score)
    
    c+=1
    if(c % 100000 == 0 ):
        ending = datetime.now()
        interval = ending - starting
        interval = interval.total_seconds()
        jam = interval // 3600
        interval %= 3600
        menit = interval // 60
        interval %= 60
        detik = interval
        print("{} durasinya {} jam {} menit {} detik".format(c,jam,menit,detik))

print("accuracy bernoulli ",max(score_bernoulli)," fitur ",com_col[score_bernoulli.index(max(score_bernoulli))],sep='')
print("accuracy multinomial ",max(score_multi)," fitur ",com_col[score_multi.index(max(score_multi))],sep='')
print("accuracy gaussian ",max(score_gaussian)," fitur ",com_col[score_gaussian.index(max(score_gaussian))],sep='')

In [276]:
X_ku = df2.drop("infectionProb",axis=1)
y_ku = df2["infectionProb"]
X_train,X_test,y_train,y_test = train_test_split(X_ku[['fever', 'bodyPain', 'runnyNose']],y_ku,test_size=0.2,random_state=42)

model = MultinomialNB()
model.fit(X_train,y_train)
y_predict = model.predict(X_test)
f_score = accuracy_score(y_test, y_predict)
print(f_score)

In [277]:
X = df2.drop(["infectionProb"],axis=1)
y = df2["infectionProb"]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
model = MultinomialNB()
model.fit(X_train,y_train)

y_predict = model.predict(X_test)
acc_score = accuracy_score(y_test,y_predict)
print(acc_score)

In [278]:
df

In [279]:
df2 = pd.read_csv("../input/covid19-symptoms-dataset/covid-19 symptoms dataset.csv")
df2 = df2.drop_duplicates()

df2.loc[df2['age']   <   60 , 'age'] = 0
df2.loc[df2['age']   >=  60, 'age'] = 1
df2.loc[df2['fever'] <  100 , 'fever'] = 0
df2.loc[df2['fever'] >= 100, 'fever'] = 1
df2.head()

In [280]:
df2 = pd.get_dummies(df2,columns=['diffBreath'])
df2.head()

In [281]:
X = df2.drop(["infectionProb"],axis=1)
y = df2["infectionProb"]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
model = BernoulliNB()
model.fit(X_train,y_train)

y_predict = model.predict(X_test)
acc_score = accuracy_score(y_test,y_predict)
print(acc_score)

In [282]:
df2['age'] = StandardScaler().fit_transform(df2['age'].values.reshape((-1,1)))
qq = QuantileTransformer(n_quantiles=1000,output_distribution="normal")
df2['age'] = qq.fit_transform(df2['age'].values.reshape((-1,1)))
df2['age'] = pd.cut(df2['age'],10,labels=[i for i in range(10)])
df2['age'] = df2['age'].astype('float')
df2['age'] = StandardScaler().fit_transform(df2['age'].values.reshape((-1,1)))
sns.histplot(x='age',data=df2,bins=10)

In [283]:
print(df2['fever'].value_counts())
sns.histplot(x='fever',data=df2)

In [284]:
# df2['fever'] = StandardScaler().fit_transform(df2['fever'].values.reshape((-1,1)))
# qq = QuantileTransformer(n_quantiles=1000,output_distribution="normal")
# df2['fever'] = qq.fit_transform(df2['fever'].values.reshape((-1,1)))
# df2['fever'] = pd.cut(df2['fever'],3,labels=[i for i in range(3)])
# df2['fever'] = df2['fever'].astype('float')
# df2['fever'] = StandardScaler().fit_transform(df2['fever'].values.reshape((-1,1)))
# sns.histplot(x='fever',data=df2,bins=5)

In [285]:
X_allin = df2.drop("infectionProb",axis=1)
y_allin = df2["infectionProb"]
X_train,X_test,y_train,y_test = train_test_split(X_allin,y_allin,test_size=0.2,random_state=42)
model = GaussianNB()
model.fit(X_train,y_train)
y_predict = model.predict(X_test)
f_score = accuracy_score(y_test, y_predict)
print(f_score)

In [286]:
cek_quan = []
for j in range(3,1001):
    df2 = pd.read_csv("../input/covid19-symptoms-dataset/covid-19 symptoms dataset.csv")
    df2 = df2.drop_duplicates()
    df2['diffBreath'] += 1
    df2['fever'] = StandardScaler().fit_transform(df2['fever'].values.reshape((-1,1)))
    qq = QuantileTransformer(n_quantiles=1000,output_distribution="normal")
    df2['fever'] = qq.fit_transform(df2['fever'].values.reshape((-1,1)))
    df2['fever'] = pd.cut(df2['fever'],j,labels=[i for i in range(j)])
    df2['fever'] = df2['fever'].astype('float')
    df2['fever'] = StandardScaler().fit_transform(df2['fever'].values.reshape((-1,1)))
    X_allin = df2.drop("infectionProb",axis=1)
    y_allin = df2["infectionProb"]
    X_train,X_test,y_train,y_test = train_test_split(X_allin,y_allin,test_size=0.2,random_state=42)
    model = GaussianNB()
    model.fit(X_train,y_train)
    y_predict = model.predict(X_test)
    f_score = accuracy_score(y_test, y_predict)
    cek_quan.append(f_score)
#     print(j,end=',')
# print()
print("letaknya",end=' ')
print(cek_quan.index(max(cek_quan))+3)
print(max(cek_quan))
for i in range(len(cek_quan)):
    if(max(cek_quan) == cek_quan[i]):
        print('maxing ',i+3)
plt.figure(figsize=(22, 8))
# plt.scatter([i for i in range(len(cek_quan))],cek_quan)
plt.scatter([i for i in range(10)],cek_quan[:10])

In [287]:
#bodyPain	age	runnyNose	diffBreath
selection = ['bodyPain',"runnyNose","diffBreath"]
for i in range(len(selection)):
    for j in range((i+1),len(selection)):
        if(i!=j):
            has = chi2(df2[[selection[i]]],df2[[selection[j]]])
            print("nilai p_value ",selection[i],"dan",selection[j],"adalah",has[1][0])

In [288]:
df2 = pd.get_dummies(df2,columns=['diffBreath'])

from optbinning import OptimalBinning
optb = OptimalBinning(name='age', dtype="numerical", solver="mip")

x = df2.age.values
y = df2.infectionProb
optb.fit(x, y)
optb.splits

In [289]:
X_allin = df2.drop("infectionProb",axis=1)
y_allin = df2["infectionProb"]

mi_score = mutual_info_classif(X_allin,y_allin,random_state=42)
fitur = dict(zip(X_allin.columns.tolist(),mi_score))
print(fitur)

X_train,X_test,y_train,y_test = train_test_split(X_allin,y_allin,test_size=0.2,random_state=42)
model = GaussianNB()
model.fit(X_train,y_train)
y_predict = model.predict(X_test)
f_score = accuracy_score(y_test, y_predict)

print('sebelum_seleksi',f_score)

featureku = []
for k,v in fitur.items():
    if(v!=0.0):
        featureku.append(k)

X_train,X_test,y_train,y_test = train_test_split(X_allin[featureku],y_allin,test_size=0.2,random_state=42)
model = GaussianNB()
model.fit(X_train,y_train)
y_predict = model.predict(X_test)
f_score = accuracy_score(y_test, y_predict)
print('sesudah_seleksi',f_score)

# batasan

In [290]:
X_allin = df2[['fever', 'bodyPain', 'age']]
y_allin = df2["infectionProb"]

In [291]:
x = df2.age.values
y = df2.infectionProb
optb.fit(x, y)

batasan_umur = optb.splits.tolist()

print("batasan umurnya adalah",batasan_umur)
binning = [0]
for i in batasan_umur:
    if(isinstance(x, str)):
        cek = int(i[4:6])
    else:
        cek = i
    binning.append(cek)
binning.append(100)
binning = sorted(binning)
labelku = [i for i in range(len(batasan_umur)+1)]

df2['aging'] = pd.cut(df2['age'],bins=binning,labels=labelku)
df2.drop('age',axis=1,inplace=True)
df2 = pd.get_dummies(df2,columns=['aging'])
df2.head()

age 0 == 0 - 10.5 <br>
age 1 == 10.5 -  37.5 <br> 
age 2 == 37.5, - 51.5 <br>
age 3 == 51.5, - 66.5 <br>
age 4 == 66.5, - 77.5 <br>
age 5 == 77.5 -  95.5 <br>
age 6 == 95.5 - 100

In [292]:
agingku = [('aging_'+str(i)) for i in range(7) ]
listku = [df2[i].sum() for i in agingku]
print(listku)

agingku_0 = df2.loc[df2['aging_0']==1,'infectionProb']
agingku_1 = df2.loc[df2['aging_1']==1,'infectionProb']
agingku_2 = df2.loc[df2['aging_2']==1,'infectionProb']
agingku_3 = df2.loc[df2['aging_3']==1,'infectionProb']
agingku_4 = df2.loc[df2['aging_4']==1,'infectionProb']
agingku_5 = df2.loc[df2['aging_5']==1,'infectionProb']
agingku_6 = df2.loc[df2['aging_6']==1,'infectionProb']


plt.bar(agingku,listku)

terinfeksi = [agingku_0[agingku_0==0].count(),agingku_1[agingku_1==0].count(),agingku_2[agingku_2==0].count(),
       agingku_3[agingku_3==0].count(),agingku_4[agingku_4==0].count(),agingku_5[agingku_5==0].count(),
       agingku_6[agingku_6==0].count()]
tidak_terinfeksi = (np.array(listku) - np.array(terinfeksi)).tolist()
print(terinfeksi)
print(tidak_terinfeksi)


width = 0.35


x = np.arange(len(listku))

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, terinfeksi, width, label='terinfeksi')
rects2 = ax.bar(x + width/2, tidak_terinfeksi, width, label='tidak_terinfeksi')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('banyak orang')
ax.set_ylabel('kategori umur')
ax.set_title('banyak orang terinfeksi berdasar umur')
ax.legend()
ax.set_xticks(x)
ax.set_xticklabels(agingku)


ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()

In [293]:
# selection = df2.columns.values
# for i in range(len(selection)):
#     for j in range((i+1),len(selection)):
#         if(i!=j):
#             has = chi2(df2[[selection[i]]],df2[[selection[j]]])
#             print("nilai p_value ",selection[i],"dan",selection[j],"adalah",has[1][0])

In [294]:
X_allin = df2.drop("infectionProb",axis=1)
y_allin = df2["infectionProb"]
mi_score = mutual_info_classif(X_allin,y_allin,random_state=42)
fitur_mi = dict(zip(X_allin.columns.tolist(),mi_score))

featureku = []
for k,v in fitur_mi.items():
    if(v!=0.0):
        featureku.append(k)

In [295]:
X_train,X_test,y_train,y_test = train_test_split(X_allin[featureku],y_allin,test_size=0.2,random_state=42)
model = GaussianNB()
model.fit(X_train,y_train)
y_predict = model.predict(X_test)
f_score = accuracy_score(y_test, y_predict)
print('sesudah_seleksi',f_score)
print(featureku)

In [296]:
fiturku = featureku[:len(featureku)]
nilai_akurasi = []
for i in range(1,len(fiturku) + 1):
    X_allin = df2[fiturku]
    y_allin = df2["infectionProb"]
    X_train,X_test,y_train,y_test = train_test_split(X_allin,y_allin,test_size=0.2,random_state=42)
    model = GaussianNB()
    model.fit(X_train,y_train)
    y_predict = model.predict(X_test)
    f_score = accuracy_score(y_test, y_predict)
    nilai_akurasi.append(f_score)
    fiturku.pop()
    print(i,". akurasinya ",f_score,sep='')
    print("   jumlah barisnya sebanyak ",X_test.shape[0]," baris",sep='')
    print("  ",X_allin.columns)

In [297]:
nilai_akurasi

In [298]:
indek_besar = (nilai_akurasi.index(max(nilai_akurasi)))
fiturku     = featureku[: (len(nilai_akurasi)-indek_besar)]
fiturku

In [299]:
# n_score = dict()

# for kk in range(5,2347):
#     df = pd.read_csv("../input/covid19-symptoms-dataset/covid-19 symptoms dataset.csv")
#     df = df.drop_duplicates()
#     df = df.sample(n=kk,random_state=42)
#     optb = OptimalBinning(name='age', dtype="numerical", solver="mip")
#     x = df.age.values
#     y = df.infectionProb
#     optb.fit(x, y)
#     batasan_umur = optb.splits.tolist()
#     binning = [0]
#     for i in batasan_umur:
#         if(isinstance(x, str)):
#             cek = int(i[4:6])
#         else:
#             cek = i
#         binning.append(cek)
#     binning.append(100)
#     binning = sorted(binning)
#     labelku = [i for i in range(len(batasan_umur)+1)]

#     df['aging'] = pd.cut(df['age'],bins=binning,labels=labelku)
#     df.drop('age',axis=1,inplace=True)
#     df = pd.get_dummies(df,columns=['aging','diffBreath'])
#     df = df.drop_duplicates()
#     X = df.drop("infectionProb",axis=1)
#     y = df["infectionProb"]
#     X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
#     model = Gauss()
#     model.fit(X_train,y_train)
#     y_predict = model.predict(X_test)
#     f_score = accuracy_score(y_test, y_predict)
#     f_score
#     l = []
#     for i in X_train.columns:
#         feat_chos = i
#         model.fit(X_train[[feat_chos]],y_train)
#         y_predict = model.predict(X_test[[feat_chos]])
#         f_score = accuracy_score(y_test, y_predict)
#         if(f_score>=0.5):
#             l.append(i)
#     if(l!=[]):
#         X = df[l]
#         y = df["infectionProb"]
#         X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
#         model = Gauss()
#         model.fit(X_train,y_train)
#         y_predict = model.predict(X_test)
#         f_score = accuracy_score(y_test, y_predict)
#         if(f_score>=0.7):
#             n_score[kk] = f_score
#         print(kk,'.',f_score)
#     else:
#         print(kk)

In [300]:
# print(n_score)

In [301]:
# for k,v in n_score.items():
#     if(v>=0.8):
#         print(k,'.',v)
    


In [302]:
from datetime import datetime
ending = datetime.now()
interval = ending - starting
interval = interval.total_seconds()

jam = interval // 3600
interval %= 3600
menit = interval // 60
interval %= 60
detik = interval

print("durasinya {} jam {} menit {} detik".format(jam,menit,detik))